# General

Things to fix or add:

Fourier transform of the Hahn Echo

Check Hahn Echo in qudi

Spin density in XY8

confocal imaging, ODMR

## Library import

Importing libraries and qudi_analysis where main constants and functions are defined.

In the data import of pulsed measurements the second argument is used to extract the data with an alternating signal (number of columns).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams.update({'font.size': 15})
from scipy.optimize import curve_fit
from scipy.fft import rfft, rfftfreq
import plotly.express as px
import os

import qudi_analysis_functions as qa

# XY8
## Experimental data and estimation

### Data import

Uploading qudi .dat format data files for the analysis. Change directory and filenames of the data.

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\439 G\Rabi")
# Rabi
filename = r"20221012-1221-49_Sample 19_LiF_NV3_Rabi_439G_pulsed_measurement.dat"
Rabi_time, Rabi_signal = qa.qudi_reader(filename , 2)


os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\439 G\XY8-6")
# XY8-N; Region of interest
filename=r"20221013-1157-11_Sample 19_LiF_NV3_XY8-6_Li1_439G_pulsed_measurement.dat"
XY8_roi_time, XY8_roi_signal, XY8_roi_signal_alt, XY8_roi_delta = qa.qudi_reader(filename, 3)

If the region of interest area was measured by halves.

In [ ]:
#### Append the array
# XY8-N; Region of interest
filename2=r"20221013-1735-20_Sample 19_LiF_NV3_XY8-6_Li2_439G_pulsed_measurement.dat"
XY8_roi_time_2, XY8_roi_signal_2, XY8_roi_signal_alt_2, XY8_roi_delta_2 = qa.qudi_reader(filename2, 3)
XY8_roi_time = np.append(XY8_roi_time, XY8_roi_time_2)
XY8_roi_signal = np.append(XY8_roi_signal, XY8_roi_signal_2)
XY8_roi_signal_alt = np.append(XY8_roi_signal_alt, XY8_roi_signal_alt_2)
XY8_roi_delta = np.append(XY8_roi_delta, XY8_roi_delta_2)

### Variables for the analysis

Values to change:

Sample_name: Uneder this name data files will be saved.

N: Number of XY8 blocks

nor: Number of resonance peaks.

ODMR_left & ODMR_right: ODMR dips from the qudi fit.

amplitude, freq, offset: Rabi fitting values.

gamma & gamma2 (optionally): Measured isotop(s)
reduced gyromagnetic ratio(s).

gamma_name & gamma2_name (optionally): Measured isotop(s) reduced
gyromagnetic ratio(s).

In [ ]:
Sample_name = 'Sample 19 LiF NV3 Li XY8-6 439G'

# Number of XY8 blocks
N = 6

# Number of resonances
nor = 1 # 1 or 2

# Magnetic field (use values from the qudi ODMR fitting)
ODMR_left = 2870 * 1e6   #Hz
ODMR_right = 4000 * 1e6    #Hz
B_ext = (ODMR_right - ODMR_left)/ (2 * qa.gamma_NV)    #G
# B_ext = 412   # G

# Rabi fitting (use values from the qudi .png file)
amplitude = 0.15    # a.u.
freq = 15   # MHz
offset = 0.9    # a.u.
phase = 0    # rad

# Select isotopes 
gamma = qa.gamma_13C    # Hz/G
gamma_name = '13C'
if nor == 2: # Second isotope
    gamma2 = qa.gamma_19F    # Hz/G
    gamma2_name = '19F'
# else:
#     gamma2 = gamma
#     gamma2_name = gamma_name

In [ ]:
print(B_ext)
print(gamma)

### Larmor frequency and tau spacing

qa.Estimation will print calulated gyromagnetic ratio, Larmor frequency and an expected value of tau spacing for the XY8 measurements.

In [ ]:
# Second isotope will be with lower gyromagnetic ratio
if nor == 2: 
    if gamma2 > gamma:
        gamma, gamma2 = gamma2, gamma
        gamma_name, gamma2_name = gamma2_name, gamma_name
    tau, Larm_freq, tau2, Larm_freq2 = qa.Estimation(nor, B_ext, gamma, gamma2, gamma_name, gamma2_name)
else:
    tau, Larm_freq = qa.Estimation(nor, B_ext, gamma, gamma, gamma_name, gamma_name)

## Data analysis

### Normalization and spectral density

Values to change:

skip_points_start: If XY8_roi_time starts from 0 or really small values, it is better to start from higher tau values for the power spectral density calculation.

skip_points_end: To remove some noise.

error_fix: If there is an error during spectral density calculation due to overlap of the main and  the alternating signals.

In [ ]:
skip_points_start = 0
skip_points_end = 11
error_fix = 0 # increase with 0.01 step

# Rabi fitting
Rabi_p0 = np.array([amplitude, freq, phase, offset]) 
Rabi_popt, Rabi_pcov = curve_fit(qa.cos_func, Rabi_time, Rabi_signal, Rabi_p0)

# XY8 normalization
level = ((XY8_roi_signal + XY8_roi_signal_alt)/2).mean()
XY8_norm = qa.norm(XY8_roi_signal, qa.cos_func(Rabi_time, *Rabi_popt), level)
XY8_norm_alt = qa.norm(XY8_roi_signal_alt, qa.cos_func(Rabi_time, *Rabi_popt), level)

# XY8 mean delta of 2 signals
XY8_delta = (XY8_norm + (1 - XY8_norm_alt)) / 2

# Power spectral density calculation
skip_points_end = len(XY8_roi_time) - skip_points_end
if skip_points_start != 0 or  skip_points_end != 0:
    # Calculates frequency Nu for given values of tau, kHz
    XY8_frequency = 1000 / (2 * XY8_roi_time[skip_points_start:skip_points_end])
    # In original script instead of np.max(Rabi_signal) + error_fix was just 1 which gives errors if signals are overlapping
    Spectral_dens = qa.deconv(8 * N, np.max(Rabi_signal) + error_fix - XY8_delta[skip_points_start:skip_points_end], XY8_roi_time[skip_points_start:skip_points_end])
else:
    # Calculates frequency Nu for given values of tau, kHz
    XY8_frequency = 1000 / (2 * XY8_roi_time)    
    Spectral_dens = qa.deconv(8 * N, np.max(Rabi_signal) + error_fix - XY8_delta, XY8_roi_time)

### Plotting Rabi, XY8 signal and spectral density

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (20,13))

axs[0,0].plot(Rabi_time, Rabi_signal)
axs[0,0].set(xlabel = 'Tau (ns)', ylabel = 'Florescence (a.u.)', title = 'NV Rabi')
axs[0,0].plot(Rabi_time, qa.cos_func(Rabi_time, *Rabi_popt), 'r--', label='fit:\nampl=%5.3f \nfreq=%5.3f MHz \nphase=%5.3f \noffset=%5.3f' % tuple(Rabi_popt))
axs[0,0].legend()

axs[0,1].plot(XY8_roi_time, XY8_roi_signal, label = 'pi/2')
axs[0,1].plot(XY8_roi_time, XY8_roi_signal_alt, label = '3pi/2')
axs[0,1].set(xlabel = 'Tau (\u03BCs)', ylabel = 'Florescence (a.u.)', title = 'NV XY8-' + str(N))
axs[0,1].plot(XY8_roi_time, qa.cos_func(XY8_roi_time, *Rabi_popt), 'r--', label='Rabi')
axs[0,1].legend()

axs[1,0].plot(XY8_roi_time, XY8_norm, label = 'pi/2')
axs[1,0].plot(XY8_roi_time, XY8_norm_alt, label = '3pi/2')
axs[1,0].plot(XY8_roi_time, XY8_delta, '--', label = 'Mean delta')
axs[1,0].set(xlabel = 'Tau (\u03BCs)', ylabel = 'Florescence (norm.)', title = 'NV XY8-'+ str(N) + ' normalized data')
# Expected sensing spacing
axs[1,0].axvline(x = tau, color = 'grey', linestyle = '--')
axs[1,0].text(tau, np.max(XY8_norm_alt), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[1,0].axvline(x = tau2, color = 'olive', linestyle = '--')
    axs[1,0].text(tau2, np.max(XY8_norm_alt) - 0.1, gamma2_name, color = 'olive', fontsize = 15)
axs[1,0].legend()

axs[1,1].plot(XY8_frequency, Spectral_dens, 'g', label = 'Mean delta')
axs[1,1].set(xlabel = 'Frequency (kHz)', ylabel = 'Power spectral density (kHz)', title = 'NV spectral density')
# Expected Larmor frequencies
axs[1,1].axvline(x = Larm_freq, color = 'grey', linestyle = '--')
axs[1,1].text(Larm_freq, np.max(Spectral_dens), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[1,1].axvline(x = Larm_freq2, color = 'olive', linestyle = '--')
    axs[1,1].text(Larm_freq2, np.max(Spectral_dens) - 50, gamma2_name, color = 'olive', fontsize = 15)
axs[1,1].legend()

plt.savefig(Sample_name + " spectral density.png")

In [ ]:
# spectra_XY8 = zip(XY8_frequency, Spectral_dens)
# spectra_csv = list(spectra_XY8)

In [ ]:
# np.savetxt("Sample 19 LiF NV10 XY8-8.csv", spectra_csv, delimiter=",")

In [ ]:
# os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\400 G\XY8-2")
# with open('Sample 19 LiF NV1 XY8-2', 'w', newline = '') as csv_file:
#     writer = csv.writer(csv_file)
#     writer.writerow(XY8_frequency)
#     writer.writerow(Spectral_dens)

### XY8-N comparison

Use this part to compare XY8-N signals for different N.

In [ ]:
if N == 2:
    XY8_2_roi_time = XY8_roi_time
    XY8_2_delta = XY8_delta
    XY8_2_frequency = XY8_frequency
    XY8_2_spectral_dens = Spectral_dens
    
if N == 4:
    XY8_4_roi_time = XY8_roi_time
    XY8_4_delta = XY8_delta
    XY8_4_frequency = XY8_frequency
    XY8_4_spectral_dens = Spectral_dens

if N == 6:
    XY8_6_roi_time = XY8_roi_time
    XY8_6_delta = XY8_delta
    XY8_6_frequency = XY8_frequency
    XY8_6_spectral_dens = Spectral_dens
    
if N == 8:
    XY8_8_roi_time = XY8_roi_time
    XY8_8_delta = XY8_delta
    XY8_8_frequency = XY8_frequency
    XY8_8_spectral_dens = Spectral_dens

In [ ]:
# #### Data to compare
# XY8_6_roi_time = XY8_roi_time
# XY8_6_delta = XY8_delta
# XY8_6_frequency = XY8_frequency
# XY8_6_spectral_dens = Spectral_dens

fig, axs = plt.subplots(2, 1, figsize = (20,13))

axs[0].plot(XY8_2_roi_time, XY8_2_delta, label = 'XY8-2')
axs[0].plot(XY8_4_roi_time, XY8_4_delta, label = 'XY8-4')
axs[0].plot(XY8_6_roi_time, XY8_6_delta, label = 'XY8-6')
axs[0].plot(XY8_8_roi_time, XY8_8_delta, label = 'XY8-8')
axs[0].set(xlim=(0.5, 0.8), xlabel = 'Tau (\u03BCs)', ylabel = 'Florescence (norm.)', title = 'NV3 XY8-N normalized data')
# Expected sensing spacing
axs[0].axvline(x = tau, color = 'grey', linestyle = '--')
axs[0].text(tau, np.max(XY8_2_delta), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[0].axvline(x = tau2, color = 'olive', linestyle = '--')
    axs[0].text(tau2, np.max(XY8_2_delta) - 0.1, gamma2_name, color = 'olive', fontsize = 15)
axs[0].legend()

axs[1].plot(XY8_2_frequency, XY8_2_spectral_dens, label = 'XY8-2')
axs[1].plot(XY8_4_frequency, XY8_4_spectral_dens, label = 'XY8-4')
axs[1].plot(XY8_6_frequency, XY8_6_spectral_dens, label = 'XY8-6')
axs[1].plot(XY8_8_frequency, XY8_8_spectral_dens, label = 'XY8-8')
axs[1].set(xlim=(635, 1000),xlabel = 'Frequency (kHz)', ylabel = 'Power spectral density (kHz)', title = 'NV3 spectral density')
# Expected Larmor frequencies
axs[1].axvline(x = Larm_freq, color = 'grey', linestyle = '--')
axs[1].text(Larm_freq, np.max(XY8_2_spectral_dens), gamma_name, color = 'grey', fontsize = 15)
if nor == 2 and tau != tau2:
    axs[1].axvline(x = Larm_freq2, color = 'olive', linestyle = '--')
    axs[1].text(Larm_freq2, np.max(XY8_2_spectral_dens) - 50, gamma2_name, color = 'olive', fontsize = 15)
# axs[1].set_yscale('log')
axs[1].legend()

plt.savefig("Sample 19 LiF NV3 XY8-N Li 439 G spectral density.png")

### Interactive plot

Change signal and time accordingly. You can use this plot to determine peak frequencies, amplitudes and FWHM for the fitting.

In [ ]:
signal = Spectral_dens
time = XY8_frequency


figexp = px.line(x = time, y = signal, title="FT")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text = "Frequency (MHz)")
figexp.update_yaxes(title_text = "Amplitude(arb. units)")
figexp.show(config = config)

### Lorentzian fitting

Values to change:

amplitude, center, sigma, baseline: Lorentzian fitting values, use interactive plot to correct values.

center2: Depending on the highest peak position you need to either sum or substract.

In [ ]:
# nor = 1 # number of resonances

In [ ]:
amplitude = np.max(Spectral_dens)
center = XY8_frequency[np.argmax(Spectral_dens)]
sigma = 30
baseline = 40

if nor == 2:
    center2 = center + (Larm_freq - Larm_freq2) # OR = center - (Larm_freq - Larm_freq2)
    Lorentzian_p0 = np.array([amplitude, center, sigma, amplitude, center2, sigma, baseline])
#     Lorentzian_p0 = np.array([400, 1900, 50, 110, 1785, 60, 0]) # if fitting doesn't work use manual values
    Spectrum_fit, pcov = curve_fit(qa.double_Lorentzian_func, XY8_frequency, Spectral_dens, Lorentzian_p0)
else:
    Lorentzian_p0 = np.array([amplitude, center, sigma, baseline])
#     Lorentzian_p0 = np.array([89, 675, 20, 0]) # if fitting doesn't work use manual values
    Spectrum_fit, pcov = curve_fit(qa.Lorentzian_func, XY8_frequency, Spectral_dens, Lorentzian_p0)
    
decay_check = 0    
print(Lorentzian_p0)
print(Spectrum_fit)

#### Slopy baseline

Adding linear function for better fitting of data with slopy baseline. Use either this or previous cell.

In [ ]:
amplitude = np.max(Spectral_dens)
center = XY8_frequency[np.argmax(Spectral_dens)]
sigma = 30
baseline = Spectral_dens[0]
decay = Spectral_dens[-1]

if nor == 2:
    center2 = center + (Larm_freq - Larm_freq2) # OR = center - (Larm_freq - Larm_freq2)
    amplitude2 = amplitude
    Lorentzian_p0 = np.array([amplitude, center, sigma, amplitude, center2, sigma, decay, baseline])
    Lorentzian_p0 = np.array([265, 1637, 11, 190, 1769, 28, 0, 0])
    Spectrum_fit, pcov = curve_fit(qa.double_Lorentzian_decay_func, XY8_frequency, Spectral_dens)
else:
    Lorentzian_p0 = np.array([amplitude, center, sigma, decay, baseline])
#     Lorentzian_p0 = np.array([89, 730, 10, 0, 0])
    Spectrum_fit, pcov = curve_fit(qa.Lorentzian_decay_func, XY8_frequency, Spectral_dens,Lorentzian_p0)

decay_check = 1
print(Lorentzian_p0)
print(Spectrum_fit)

### Spectral density plotting

In [ ]:
fig, axs = plt.subplots(figsize = (18,10))
axs.plot(XY8_frequency, Spectral_dens, 'o',label = 'Delta')
if decay_check == 0:
    if nor == 2:
        axs.plot(XY8_frequency, qa.double_Lorentzian_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \namplitude2=%5.3f \ncenter2=%5.3f  \nsigma2=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
    else:
        axs.plot(XY8_frequency, qa.Lorentzian_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
else:
    if nor == 2:
        axs.plot(XY8_frequency, qa.double_Lorentzian_decay_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \namplitude2=%5.3f \ncenter2=%5.3f  \nsigma2=%5.3f \ndecay=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
    else:
        axs.plot(XY8_frequency, qa.Lorentzian_decay_func(XY8_frequency, *Spectrum_fit), 'r--', label='fit:\namplitude=%5.3f \ncenter=%5.3f  \nsigma=%5.3f \ndecay=%5.3f \noffset=%5.3f' % tuple(Spectrum_fit))
        
axs.legend()
axs.set(xlabel = 'Frequency (kHz)', ylabel = 'Power spectral density (kHz)', title = 'Noise spectral density')

plt.savefig(Sample_name + " spectrum.png")

### Expreimental Larmor frequency and magnetic field

Prints difference of experimental values with the theoretical calculations.

In [ ]:
gamma = gamma2
gamma_name = gamma2_name

In [ ]:
# Experimental Larmor frequency and magnetic field
peak_freq = Spectrum_fit[1] # peak_freq should be higher peak_freq2!
B_calc = peak_freq * 1e3 / gamma

# Checking that fitted peaks relative positions are same as theoretical
if nor == 2:
    peak_freq2 = Spectrum_fit[4]
    if peak_freq < peak_freq2:
        peak_freq, peak_freq2 = peak_freq2, peak_freq
        B_calc = peak_freq * 1e3 / gamma
        B_calc2 = peak_freq2 * 1e3 / gamma2
        check = 1
    else:
        B_calc2 = peak_freq2 * 1e3 / gamma2
        check = 0
    print(gamma_name + ' Larmor frequency from the experimental data:', round(peak_freq,2), 'kHz')
    print(gamma_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq - Larm_freq,2), 'kHz')
    print(gamma_name + ' magnetic field from the experimental data:', round(B_calc,2), 'G')
    print(gamma_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc - B_ext,2), 'G')
    print('\nFor the second signal:')
    print(gamma2_name + ' Larmor frequency from the experimental data:', round(peak_freq2,2), 'kHz')
    print(gamma2_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq2 - Larm_freq2,2), 'kHz')
    print(gamma2_name + ' magnetic field from the experimental data:', round(B_calc2,2), 'G')
    print(gamma2_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc2 - B_ext,2), 'G')
else:
    print(gamma_name + ' Larmor frequency from the experimental data:', round(peak_freq,2), 'kHz')
    print(gamma_name + ' difference between theoretical Larmor frequency and measured one:', round(peak_freq - Larm_freq,2), 'kHz')
    print(gamma_name + ' magnetic field from the experimental data:', round(B_calc,2), 'G')
    print(gamma_name + ' difference between magnetic field projection on the NV and measured one:', round(B_calc - B_ext,2), 'G')

### NMR signal

Calculates Lorentzian linewidth and magnetic field fluctuations.

In [ ]:
# Getting the integral of the magnetic noise spectrum with substracted offset
spectrum_area = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[0:3], Spectrum_fit[3]) - Spectrum_fit[3], x = np.sort(XY8_frequency))/ 1e6    # MHz^2
# Lorentzian linewidth with the substracted offset
linewidth = 2 * np.abs(Spectrum_fit[2])
if nor == 2:
    # Getting the integral of the magnetic noise spectrum with substracted offset
    if decay_check == 1: #######CHECK substarction, it is not correct##########
        spectrum_area2 = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[3:-1]) - Spectrum_fit[6], x = np.sort(XY8_frequency))/ 1e6     #MHz^2
        # Lorentzian linewidth with the substracted offset
        linewidth2 = 2 * np.abs(Spectrum_fit[5])
    else:
        spectrum_area2 = np.trapz(qa.Lorentzian_func(XY8_frequency, *Spectrum_fit[3:]) - Spectrum_fit[6], x = np.sort(XY8_frequency))/ 1e6     #MHz^2
        # Lorentzian linewidth with the substracted offset
        linewidth2 = 2 * np.abs(Spectrum_fit[5])
    if check == 1:
        spectrum_area, spectrum_area2 = spectrum_area2, spectrum_area
        linewidth, linewidth2 = linewidth2, linewidth
    print(gamma_name + ' Lorentzian linewidth:', round(linewidth,2), 'kHz')
    print(gamma2_name + ' Lorentzian linewidth:', round(linewidth2,2), 'kHz')
else:
    print(gamma_name + ' Lorentzian linewidth:', round(linewidth,2), 'kHz')
    
# The magnetic field fluctuations (NMR magnetic signal)
B1 = (2 * spectrum_area / qa.gamma_e**2)**0.5    # T
print(gamma_name + ' B1 =', round(B1 * 1e6,2), '\u03BCT')
if nor == 2:
    B2 = (2 * spectrum_area2 / qa.gamma_e**2)**0.5    # T
    print(gamma2_name + ' B2 =', round(B2 * 1e6,2), '\u03BCT')

### NV center depth

Calculating the depth of the NV in the case of the hydrogen spins in immersion oil.

In [ ]:
depth1 = qa.depth(B1, gamma * 1e4, qa.rho_oil)
print('depth1 =', depth1, 'nm')
if nor == 2:
    depth2 = qa.depth(B2, gamma * 1e4, qa.rho_oil)
    mean_depth = (depth1 + depth2) / 2
    print('depth2 =', depth2, 'nm')
    print('mean depth =', mean_depth, 'nm')

In [ ]:
## calcuate the spin density for given depth:

In [ ]:

# d1 = 3.7027003282742967
# d2 = 3.7027003282742967
# rho1 = dc.calculate_density_simple(mu_0, h, B1, gamma_p, d1)
# rho2 = dc.calculate_density_simple(mu_0, h, B2, gamma_p, d2)
# print('rho1 =', rho1, 'nm^-3')
# print('rho2 =', rho2, 'nm^-3')

# Hahn Echo

## Experimental data and estimation

### Data import

qudi for Hahn Echo measurements creates, plots and saves data with tau spacing between pulses instead of tau/2.

In [ ]:
## Check again data saving!!

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Small magnetic field\Hahn Echo")

# Hahn Echo
filename = r"20220909-1518-17_Sample 19_LiF_NV1_Hahn Echo_pulsed_measurement.dat"
T2_time, T2_signal, T2_signal_alt, T2_delta = qa.qudi_reader(filename, 3)
# qudi for Hahn Echo measurements creates, plots and saves data with tau spacing between pulses instead of tau/2.
# Therefore T2_time should be two times larger.
# T2_time = 2 * T2_time

os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Small magnetic field\Rabi")
# Rabi
filename = r"20220909-1354-06_Sample 19_LiF_NV1_Rabi_pulsed_measurement.dat"
Rabi_time, Rabi_signal = qa.qudi_reader(filename , 2)

### Variables for the analysis

In [ ]:
Sample_name = 'Sample'

# Rabi fitting (use values from the qudi .png file)
amplitude = 0.2    # a.u.
freq = 22    # MHz
offset = 0.9    # a.u.
phase = 0    # rad

## Data analysis

### Coherence time

Transforms data to correct tau spacing and calculates T2 time, fits the exponential decay.

In [ ]:
## Check how qudi saves the data!

In [ ]:
T2_popt, T2_pcov = curve_fit(qa.T2_func, T2_time, T2_delta)
print('Coherence time T2 =', round(T2_popt[1],2), 'us')

#Rabi offset
Rabi_p0 = np.array([amplitude, freq, phase, offset]) 
Rabi_popt, Rabi_pcov = curve_fit(qa.cos_func, Rabi_time, Rabi_signal, Rabi_p0)

fig, axs = plt.subplots(1, 2, figsize = (18,5))
axs[0].plot(T2_time, T2_signal, label = 'pi/2')
axs[0].plot(T2_time, T2_signal_alt, label = '3pi/2')
axs[0].plot(T2_time, T2_delta, '--', label = 'Mean delta')
axs[0].axhline(y = Rabi_popt[3], color = 'grey', linestyle = ':')
axs[0].text(0, Rabi_popt[3], 'Rabi offset', color = 'grey', fontsize = 13)
axs[0].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'NV Hahn Echo')
axs[0].legend()

axs[1].plot(T2_time, T2_delta, 'g')
axs[1].plot(T2_time, qa.T2_func(T2_time, *T2_popt), 'r--', label='fit: \nampl=%5.3f \nT2=%5.3f \noffset=%5.3f' % tuple(T2_popt))
axs[1].axhline(y = Rabi_popt[3], color = 'grey', linestyle = ':')
axs[1].text(0, Rabi_popt[3], 'Rabi offset', color = 'grey', fontsize = 13)
axs[1].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'NV coherence time')
axs[1].legend()

# fig.savefig(Sample_name + " Coherence time.png")

### Fourier transform

In [ ]:
### Check qudi FFT

In [ ]:
osc = Rabi_signal
fs = Rabi_time[1] - Rabi_time[0]    # Sampling frequency, us
fft_osc = rfft(osc)        # Real values of FFT
fft_fre = rfftfreq(len(Rabi_time), d = fs)


fig, axs = plt.subplots(1, 2, figsize = (18,5))
axs[0].plot(Rabi_time, osc)
axs[0].set(xlabel = 'Tau (us)', ylabel = 'Florescence (a.u.)', title = 'Hahn Echo without exp decay')

axs[1].plot(fft_fre, np.abs(fft_osc))
axs[1].set(xlabel = 'Frequency (MHz)', ylabel = 'Amplitude (a.u.)', title = 'NV FFT')

In [ ]:
# INTERACTIVE PLOT
fft_fre = fft_fre
fft_osc = fft_osc


figexp = px.line(x = fft_fre, y = np.abs(fft_osc), title="FT")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text = "Frequency (MHz)")
figexp.update_yaxes(title_text = "Amplitude(arb. units)")
figexp.show(config = config)

# Spectrum

## Experimantal data and estimation

### Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\SnSe2\20230601\Spectrum")

filename = "Sample17_SnSe2_background_90uW_60s.csv"
wavelength,spectrum = qa.csvReader(filename)
label1 =  filename[:-4] #'Background'

filename2 = "Sample17_SnSe2_NV1_90uW_60s.csv"
wavelength2,spectrum2 = qa.csvReader(filename2)
label2 =  filename2[:-4] #'NV #1'

filename3 = "Sample17_SnSe2_NV2_90uW_60s.csv"
wavelength3,spectrum3 = qa.csvReader(filename3)
label3 =  filename3[:-4] #'NV #2'

filename4 = "Sample17_SnSe2_NV3_90uW_60s.csv"
wavelength4,spectrum4 = qa.csvReader(filename4)
label4 =  filename4[:-4] #'NV #3'

filename5 = "Sample17_SnSe2_NV4_90uW_60s.csv"
wavelength5,spectrum5 = qa.csvReader(filename5)
label5 =  filename5[:-4] #'NV #4'

filename6 = "Sample17_SnSe2_NV5_90uW_60s.csv"
wavelength6,spectrum6 = qa.csvReader(filename6)
label6 =  filename6[:-4] #'NV #5'

filename7 = "Sample17_SnSe2_spot1_90uW_60s.csv"
wavelength7,spectrum7 = qa.csvReader(filename7)
label7 =  filename7[:-4] #'Bright spot #1'

filename8 = "Sample17_SnSe2_spot2_90uW_60s.csv"
wavelength8,spectrum8 = qa.csvReader(filename8)
label8 =  filename8[:-4] #'Bright spot #2'

# filename9 = "SnSe2_sample2_bright spot_15uW_10s.csv"
# wavelength9,spectrum9 = qa.csvReader(filename9)
# label9 =  filename9[:-4] #""

# filename10 = "MoTe2_area 1_100 uW_60 s.csv"
# wavelength10,spectrum10 = qa.csvReader(filename10)
# label10 =  filename10[:-4] #""

# background1 = "Sample 17 air objective background laser 100 uW exposure time 1 min.csv"
# wavelength_b1,spectrum_b1 = qa.csvReader(background1)
# label_b1 =  background1[:-4] #'Background'

# background2 = "Sample 17_background_110uW_60s.csv"
# wavelength_b2,spectrum_b2 = qa.csvReader(background2)
# label_b2 =  background1[:-4] #""

## Data analysis

### Interactive plot

In [ ]:
wavelength_x = wavelength
spectrum_y = spectrum4

figexp=px.line(x=wavelength_x,y=spectrum_y,title="Fluorescence spectrum")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text="Wavelength (nm)")
figexp.update_yaxes(title_text="Counts (arb. units)")
figexp.show(config=config)

### Spectrum

Plotting parameters.

In [ ]:
xmin = min(wavelength)
xmax = max(wavelength)
ymin = min(spectrum)
ymax = max(spectrum5) + 10

title = 'Sample 17 SnSe2'
xlabel = 'Wavelength (nm)'
ylabel = 'Counts (arb. units)'

Offset and background corrections.

In [ ]:
# spectrum = spectrum - spectrum_b1
# spectrum2 = spectrum2 - spectrum_b1
# spectrum3 = spectrum3 - spectrum_b2
# spectrum4 = spectrum2 - spectrum

# offset = 5
# offset2 = 30
# # offset3 = 30
# # offset4 = 40
# offset5 = 50

# spectrum = spectrum + offset
# spectrum2 = spectrum2 + offset2
# # spectrum4 = spectrum4 + offset4
# spectrum5 = spectrum5 + offset5

In [ ]:
fig, axs = plt.subplots(figsize=(18,10))

axs.plot(wavelength, spectrum, '-', color = 'tab:blue', label = label1)
axs.plot(wavelength2, spectrum2, '-', color = 'tab:green', label = label2)
axs.plot(wavelength3, spectrum3, '-', color = 'tab:orange', label = label3)
axs.plot(wavelength4, spectrum4, '-', color = 'tab:red', label = label4)
axs.plot(wavelength5, spectrum5, '-', color = 'tab:cyan', label = label5)
axs.plot(wavelength6, spectrum6, '-', color = 'tab:brown', label = label6)
axs.plot(wavelength7, spectrum7, '-', color = 'tab:olive', label = label7)
axs.plot(wavelength8, spectrum8, '-', color = 'tab:purple', label = label8)
# axs.plot(wavelength_b, spectrum_b, '-', color = 'tab:cyan', label = label_b1)

# axs.axvline(x = 650, color='grey', linestyle='--')
# axs.text(641, 650, '650 nm',color='grey',fontsize = 15)

axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
axs.grid(True)
axs.legend()
plt.show()


# fig.savefig(title + "_spectrum.png")

### Gradient spectrum

In [ ]:
num_plot = 8
palette = list(sns.color_palette('inferno', num_plot).as_hex()) # Colormaps:'viridis', 'plasma', 'inferno', 'magma', 'cividis'

In [ ]:
fig, axs = plt.subplots(figsize=(18,10))

axs.plot(wavelength, spectrum, '-', color = palette[0], label='15 \u03BCW') #'tab:blue'
axs.plot(wavelength2, spectrum2, '-', color = palette[1], label='30 \u03BCW') #'tab:green'
axs.plot(wavelength3, spectrum3, '-', color = palette[2], label='40 \u03BCW') #'tab:orange'
axs.plot(wavelength4, spectrum4, '-', color = palette[3], label='50 \u03BCW') #'tab:red'
axs.plot(wavelength5, spectrum5, '-', color = palette[4], label='60 \u03BCW') #'tab:cyan'
axs.plot(wavelength6, spectrum6, '-', color = palette[5], label='70 \u03BCW') #'tab:brown'
axs.plot(wavelength7, spectrum7, '-', color = palette[6], label='80 \u03BCW')
axs.plot(wavelength8, spectrum8, '-', color = palette[7], label='90 \u03BCW')
# axs.plot(wavelength9, spectrum9, '-', color = palette[8], label='Bright spot 15 \u03BCW')
# axs.plot(wavelength10, spectrum10, '-', color = palette[9], label='100 \u03BCW')


axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
axs.grid(True)
axs.legend()
plt.show()


# fig.savefig(title + "_gradient.png")

# Saturation

## Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 17\acid cleaning") 

filename = np.loadtxt(r'Saturation_acid.txt', delimiter=',')
laser_power = filename[:,0] # np.array of float values of the laser power
NV_counts = filename[:,1] # np.array of float values of the intensity
NV_background = filename[:,2]# np.array of float values of the intensity
NV_sub = NV_counts - NV_background

##For the comaprison with an old data##
filename2 = np.loadtxt(r'Saturation.txt', delimiter=',')
laser_power2 = filename2[:,0] # np.array of float values of the laser power
NV_counts2 = filename2[:,1] # np.array of float values of the intensity
NV_background2 = filename2[:,2]# np.array of float values of the intensity
NV_sub2 = NV_counts2 - NV_background2
####

## Data analysis

In [ ]:
saturation_popt, saturation_pcov = curve_fit(qa.saturation_func, laser_power[0:4], NV_sub[0:4])

fig, axs = plt.subplots(1,2,figsize=(18,6))

axs[0].plot(laser_power, NV_counts,'o',color='green',label='NV counts')
axs[0].plot(laser_power, NV_background,'o',color='lightgreen',label='Background counts')
##Comparison##
axs[0].plot(laser_power2, NV_counts2,'o',color='red',label='Old NV counts')
axs[0].plot(laser_power2, NV_background2,'o',color='pink',label='Old Background counts')
####
axs[0].set(xlabel='Laser power (\u03bcW)', ylabel='Intensity (arb. units)', title='Single NV counts')
axs[0].legend()
axs[0].grid(True)
axs[0].ticklabel_format(axis='y',style='sci',scilimits=(5,5))
axs[0].set(xlim=(-20, 1250))

axs[1].plot(laser_power, NV_sub,'o',color='green',label='NV counts')
axs[1].plot(laser_power, NV_background,'o',color='lightgreen',label='Background counts')
##Comparison
axs[1].plot(laser_power2, NV_sub2,'o',color='red',label='Old NV counts')
axs[1].plot(laser_power2, NV_background2,'o',color='pink',label='Old Background counts')
####
axs[1].plot(laser_power, qa.saturation_func(laser_power,*saturation_popt),'g--') #, label='fit: a=%.2e' % tuple(popt)
axs[1].set(xlabel='Laser power (\u03bcW)', ylabel='Intensity (arb. units)',title='Single NV counts without background')
axs[1].legend()
axs[1].grid(True)
axs[1].ticklabel_format(axis='y', style='sci',scilimits=(5,5))
axs[1].set(xlim=(-20, 300), ylim = (-0.1,150000))

# fig.suptitle("Sample 17 (Single NV) saturation curves")

# plt.savefig("saturation 2023.png")

# Lifetime

## Experimental data and estimation

### Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\SPV 18+19+ref\Lifetime")
    
filename = "Sample 19_NV0_1 min_480 nm_rep rate 2,89_current 100_LP550.mp"
lt_time, lt_counts = qa.mp_reader(filename)
lt_label1 = 'Excitation wavelength 480 nm'

filename2 = "Sample 19_NV_1 min_rep rate 2,89_current 100_LP650.mp"
lt_time2, lt_counts2 = qa.mp_reader(filename2)
lt_label2 = 'Excitation wavelength 520 nm'

### Variables for the analysis

Values to change:

lifetime: expected lifetime for the fitting.

decays: number of exponential decays for the fiting.

In [ ]:
Sample_name = 'Sample'
lifetime = 1e-8
decays = 1 # 1 or 2

## Data analysis

### Lifetime

In [ ]:
if decays == 1:
    print(lt_label1)
    time1, counts1, lt1, lt_popt1, lt_perr1 = qa.Lifetime_exp(lt_time, lt_counts, lifetime)
    print('Lifetime: ', round(lt1,3), "\u00B1", round((lt_perr1[1] * 1e9),3), 'ns')
    print('\n' + lt_label2)
    time2, counts2, lt2, lt_popt2, lt_perr2 = qa.Lifetime_exp(lt_time2, lt_counts2, lifetime)
    print('Lifetime: ', round(lt2,3), "\u00B1", round((lt_perr2[1] * 1e9),3), 'ns')
else:
    print(lt_label1)
    time1, counts1, lt1, lt_21, lt_popt1, lt_perr1 = qa.Lifetime_double_exp(lt_time, lt_counts, lifetime)
    print('Lifetime 1: ', round(lt1,3), "\u00B1", round((lt_perr1[1] * 1e9),3), 'ns')
    print('Lifetime 2: ', round(lt_21,3), "\u00B1", round((lt_perr1[3] * 1e9),3), 'ns')
    print('\n' + lt_label2)
    time2, counts2, lt2 ,lt_22, lt_popt2, lt_perr2 = qa.Lifetime_double_exp(lt_time2, lt_counts2, lifetime)
    print('Lifetime 1: ', round(lt2,3), "\u00B1", round((lt_perr2[1] * 1e9),3), 'ns')
    print('Lifetime 2: ', round(lt_22,3), "\u00B1", round((lt_perr2[3] * 1e9),3), 'ns')

### Interactive plot

In [ ]:
xdata = lt_time
ydata = lt_counts

figexp=px.line(x=xdata,y=ydata,title="Lifetime")
config = {
  'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },    'displaylogo': False
}
figexp.update_xaxes(title_text="Time (ps)")
figexp.update_yaxes(title_text="Counts (arb. units)")
figexp.show(config=config)

### Lifetime plot

In [ ]:
xmin = min(time1 * 1e9)
xmax = 10 #max(time1 * 1e9)
ymin = min(counts1) - 100 #min(spectrum)
ymax = max(counts1) + 100

title = 'Sample lifetime'
xlabel = 'Time (ns)'
ylabel = 'Counts'   

In [ ]:
fig, axs = plt.subplots(figsize=(18,10))
if decays == 1:
    axs.plot(time1 * 1e9, counts1, 'o', label='Excitation wavelength 480 nm')
    axs.plot(time1 * 1e9, qa.T2_func(time1, *lt_popt1), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\noffset=%i' % tuple(lt_popt1))
    axs.plot(time2 * 1e9, counts2, 'o', label='Excitation wavelength 520 nm')
    axs.plot(time2 * 1e9, qa.T2_func(time2, *lt_popt2), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\noffset=%i' % tuple(lt_popt2))
    axs.grid(color='gray', linestyle='-', linewidth=0.1)
    axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
    axs.legend()
#     fig.savefig(filename[:-3] + "_lifetime.png")
else:
    axs.plot(time1 * 1e9, counts1, 'o', label='Excitation wavelength 480 nm')
    axs.plot(time1 * 1e9, qa.double_exp(time1, *lt_popt1), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\namplitude2=%.2e\nlifetime2=%.2e\noffset=%i' % tuple(lt_popt1))
    axs.plot(time2 * 1e9, counts2, 'o', label='Excitation wavelength 520 nm')
    axs.plot(time2 * 1e9, qa.double_exp(time2, *lt_popt2), '--', linewidth=1, label='fit:\namplitude=%.2e\nlifetime=%.2e\namplitude2=%.2e\nlifetime2=%.2e\noffset=%i' % tuple(lt_popt2))
    axs.grid(color='gray', linestyle='-', linewidth=0.1)
    axs.set(xlim=(xmin, xmax),ylim=(ymin, ymax), xlabel=xlabel, ylabel=ylabel, title=title)
    axs.legend()
#     fig.savefig(filename[:-3] + "_lifetime_double exp.png")

# Confocal

## Data import

In [ ]:
os.chdir(r"D:\Kseniia\Optics Lab\Sample 19 (SPV)\LiF\Confocal")
reader = np.loadtxt(r'20220916-1301-30_confocal_xy_image_Dev1Ctr3.dat')
inputfile = open(r"20220916-1301-30_confocal_xy_image_Dev1Ctr3.dat", 'r')
import re

In [ ]:
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')
c = []

for lineIndex, line in enumerate(inputfile): # Same as "for counter, value in enumerate(my_list):"
    if lineIndex > 4 and lineIndex < 15: 
        print(line[1:])
        num = [float(x) for x in re.findall(match_number, line)]
        c.append(num[0])    

# CHANGEBLE VALUES
# To change contrast of the image, %
colorbar_min = 100
colorbar_max = 100
# Counts prefix
counts_prefix = 1e3
label_prefix = 'Mcounts/s'


xlim_min = 0
xlim_max = 200
xtick_step = (xlim_max - xlim_min) / 5
print(xtick_step)

xtick_min = c[0]*1e6
xtick_max = c[1]*1e6
xtick_labelstep = (xtick_max - xtick_min) / 5
xtick_label = np.arange(xtick_min,xtick_max+1,step=xtick_labelstep)

ylim_min = 0
ylim_max = 200
ytick_step = (ylim_max - ylim_min) / 5
print(ytick_step)

ytick_min = c[3]*1e6
ytick_max = c[4]*1e6
ytick_labelstep = (ytick_max - ytick_min) / 5
ytick_label = np.arange(ytick_min,ytick_max+1,step=ytick_labelstep)

colormap_min = np.min(counts)*colorbar_min/100
colormap_max = np.max(counts)*colorbar_max/100

counts = reader/counts_prefix # np.array of float values of the counts

plt.figure(figsize=(18, 18))
plt.imshow(counts, cmap='inferno', vmin = colormap_min, vmax = colormap_max)
plt.colorbar(label='Fluorescence (' + label_prefix + ')')

plt.xticks(np.arange(xlim_min, xlim_max+1, step=xtick_step), xtick_label)
plt.xlabel('X position (\u03bcm)')
plt.xlim(xlim_min,xlim_max)


plt.yticks(np.arange(ylim_min, ylim_max+1, step=ytick_step), ytick_label) 
plt.ylabel('Y position (\u03bcm)')
plt.ylim(ylim_min,ylim_max)

In [ ]:
res_max = np.max(counts)
res_min = np.min(counts)

print(res_max, res_min)
counts = reader.tolist()
resolution = 100

plt.figure(figsize=(12, 10))
plt.contour(counts, resolution, cmap='inferno')
plt.contourf(counts, resolution, cmap='inferno')
plt.colorbar(label='Fluorescence (counts/s)')
plt.xlabel('X position (\u03bcm)')
# plt.xlim(0,200)
plt.ylabel('Y position (\u03bcm)')
# plt.ylim(0,200)

# plt.savefig("test.pdf")